In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/78.2 GB disk)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
import cv2
import os

import re

from pathlib import Path
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from ultralytics import RTDETR


In [3]:
# specify the data splits you have present
split = {'train': True, 'val': True, 'test': False}
#Specify the directory where your images are located: )(specify absolute path to the directory)
train_img_dir = Path('/content/rt-detr/images/train')
val_img_dir = Path('/content/rt-detr/images/val')


#directory where the .txt files will be saved to
output_dir = Path(output_dir = '/content/rt-detr')

# Define the file extension pattern for your images
file_pattern = '*.png'


In [4]:
if split['train']:
    # Use the glob method of Path to find all files matching the pattern in the specified directory
    training_img_files = list(train_img_dir.glob(file_pattern))
    #the image names are of the format tile_0_0_img_2.txt, so we need to sort them based on the three numbers
    training_img_files.sort(key=lambda f: [int(x) for x in re.findall(r'\d+', str(f))])
    with open(os.path.join(output_dir, 'train.txt'), "a+") as f:
        for img in training_img_files:
            x = os.path.join(train_img_dir, img)
            f.write(x)
            print(x)
            f.write('\n')
        print("Done")
    f.close()

/content/rt-detr/images/train/tile_0_0_img_24.png
/content/rt-detr/images/train/tile_0_0_img_51.png
/content/rt-detr/images/train/tile_0_0_img_58.png
/content/rt-detr/images/train/tile_0_0_img_63.png
/content/rt-detr/images/train/tile_0_0_img_64.png
/content/rt-detr/images/train/tile_0_0_img_80.png
/content/rt-detr/images/train/tile_0_0_img_87.png
Done


In [5]:
# -------------------- Create val.txt file --------------------
if split['val']:
    val_img_files = list(val_img_dir.glob(file_pattern))
    val_img_files.sort(key=lambda f: [int(x) for x in re.findall(r'\d+', str(f))])
    with open(os.path.join(output_dir,'val.txt'), "a+") as f:
        for img in val_img_files:
            x = os.path.join(val_img_dir, img)
            f.write(x)
            print(x)
            f.write('\n')
        print("Done")
    f.close()

/content/rt-detr/images/val/tile_0_0_img_89.png
/content/rt-detr/images/val/tile_0_0_img_147.png
/content/rt-detr/images/val/tile_0_0_img_149.png
/content/rt-detr/images/val/tile_0_0_img_172.png
/content/rt-detr/images/val/tile_0_0_img_175.png
Done


In [6]:
from ultralytics import RTDETR

In [7]:
model = RTDETR('rtdetr-l.pt')

100%|██████████| 63.4M/63.4M [00:00<00:00, 224MB/s]


In [8]:
results = model.train(data='/content/rt-detr/main.yml', epochs=10, imgsz=640)

Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=/content/rt-detr/main.yml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 23.9MB/s]


Overriding model.yaml nc=80 with nc=1
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5, 

100%|██████████| 6.23M/6.23M [00:00<00:00, 110MB/s]


AMP: checks passed ✅


...

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:79.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
      10/10      6.69G      1.165      14.74     0.3914         17        640: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  8.91it/s]

                   all          5          6    0.00134      0.333    0.00104   0.000499



10 epochs completed in 0.010 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 66.1MB
Optimizer stripped from runs/detect/train/weights/best.pt, 66.1MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
rt-detr-l summary: 498 layers, 31985795 parameters, 0 gradients, 103.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.72it/s]


                   all          5          6    0.00135      0.333    0.00104   0.000499
Speed: 0.2ms preprocess, 25.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train


In [9]:
results = model('/content/rt-detr/download (1).jpg (1).png')


image 1/1 /content/rt-detr/download (1).jpg (1).png: 640x640 300 nothings, 159.5ms
Speed: 2.8ms preprocess, 159.5ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 640)
